In [1]:
import numpy as np
import numpy.random
import matplotlib.pyplot as plt

class Model:
    def __init__(self):
        self.weight = np.zeros(100)
        
    def __call__(self, x: np.ndarray):
        x = (self.weight * x).sum(axis=1, keepdims=True)
        x = 1 / (1 + np.exp(-x)) 
        return x
    
    def __str__(self):
        return f'{self.weight}'

def BCELoss(pred: np.ndarray, label: np.ndarray):
    return -(label * np.log(pred) + (1 - label) * np.log(1 - pred)).mean(dtype=np.float64)

In [ ]:
def preprocess_data(data):
    r'''
    compute input data of model base on g(x, y; \theta)
    '''
    transformed_data = data
    for i in range(10):
        for j in range(10):
            transformed_data = np.concatenate((transformed_data, data[:, 0:1] ** i * data[:, 1:2] ** j), axis=1)
    return transformed_data[:, 2:]


data = np.genfromtxt("data-nonlinear.txt", delimiter=',')
y_train = data[:, 2:3]
transformed_data = preprocess_data(data[:, 0:2])
transformed_data.shape